In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
import zipfile
# Replace 'DATASET.zip' with the actual name of your zip file
with zipfile.ZipFile('Plastic_Sign.zip', 'r') as zip_ref: #replace it with ur zipfilename
    zip_ref.extractall('DATASET') # extracts to a directory called 'DATASET'

In [3]:
# Paths to dataset folders
TRAIN_DIR = '/content/DATASET/Plastic_Sign/Train'
TEST_DIR = '/content/DATASET/Plastic_Sign/Test'
VID_DIR = '/content/DATASET/Plastic_Sign/Valid'

In [4]:
 # Image size and batch size
IMG_SIZE = (224, 224)  # Resize all images to 128x128
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'   # categorical
 )
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #categorical
 )
validation_generator = validation_datagen.flow_from_directory(
    VID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #categorical
 )


Found 685 images belonging to 8 classes.
Found 685 images belonging to 8 classes.
Found 685 images belonging to 8 classes.


In [5]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224,224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(224, activation='relu'),
        Dropout(0.5),
        Dense(8, activation='softmax')  # softmax for multi classification
    ])
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',  #sparse_categorical_crossentropy
                  metrics=['accuracy'])
    return model
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 224)                 │      19,382,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 224)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           1,800 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,477,544 (74.30 MB)

 Trainable params: 19,477,544 (74.30 MB)

 Non-trainable params: 0 (0.00 B)

## EPOCHS = 5
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator  # Corrected to use valid_generator
)

In [9]:
EPOCHS = 9
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator  # Corrected to use valid_generator
)

Epoch 1/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 188ms/step - accuracy: 0.6159 - loss: 1.1154 - val_accuracy: 0.6628 - val_loss: 0.9745
Epoch 2/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.6084 - loss: 1.1022 - val_accuracy: 0.7504 - val_loss: 0.7689
Epoch 3/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.7015 - loss: 0.8541 - val_accuracy: 0.8277 - val_loss: 0.5622
Epoch 4/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.7285 - loss: 0.7236 - val_accuracy: 0.8146 - val_loss: 0.5106
Epoch 5/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 182ms/step - accuracy: 0.7771 - loss: 0.6580 - val_accuracy: 0.8628 - val_loss: 0.4322
Epoch 6/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.8150 - loss: 0.5147 - val_accuracy: 0.8832 - val_loss: 0.3293
Epoch 7/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - accuracy: 0.8504 - loss: 0.4580 - val_accuracy: 0.9022 - val_loss: 0.3889
Epoch 8/9
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 113ms/step - accuracy: 0.8628 - loss: 0.4785 - val_accuracy: 0.9416 - val_l

In [11]:
# Saving model
model.save('plastic_sign.h5')

In [12]:
# 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8947 - loss: 0.2760
Test Accuracy: 0.903649628162384


In [15]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/plastic_sign.h5')  # Update with your model path

# Path to your custom image
image_path = "/content/IMG_6248.jpg"  # Update with your image path

# Image preprocessing function
IMG_SIZE = (224, 224)  # Must match the size used during training

def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Preprocess the image
image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)

# Class labels (ensure they match the model's training)
class_labels = ['Noplastic(8)', 'OherResins(7)', 'PEHD_highdensitypolyethylene(2)','PELD_lowdensitypolyethylene(4)','PET_polyethylene(1)','PP_polypropylene(5)','PS_polystyrene(6)','PVC__polyvinylchloride(3)']

# Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]

# Output the prediction
print(f"The image is predicted to be: {predicted_class_label}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
The image is predicted to be: PS_polystyrene(6)
